In [ ]:
import kfp
import os

In [ ]:
def set_env_var():
    import os
    import logging
    
    logging.info('The environment variable LIMITS for cpu before is: {}'.format(os.getenv('LIMITS.cpu')))
    logging.info('The environment variable REQUESTS for memory before is: {}'.format(os.getenv('REQUESTS.memory')))
    
    
    limits_cpu = str(os.getenv('LIMITS.cpu',os.getenv('limits_cpu')))
    limits_memory = str(os.getenv('LIMITS.memory',os.getenv('limits_memory')))
    
    
    requests_cpu = str(os.getenv('REQUESTS.cpu',os.getenv('requests_cpu')))
    requests_memory = str(os.getenv('REQUESTS.memory',os.getenv('requests_memory')))
    
    logging.info('The environment variable limits for cpu after is: {}'.format(limits_cpu))
    logging.info('The environment variable requests for memory after is: {}'.format(requests_memory))

In [ ]:
image_name = 'tensorflow/tensorflow:1.11.0-py3'
env_function_op = kfp.components.func_to_container_op(set_env_var,base_image=image_name)

In [ ]:
import kfp.dsl as dsl
from kubernetes.client.models import V1EnvVar

@dsl.pipeline(
  name='Env eg',
  description='pl to set env var LIMITS,REQUESTS'
)
def environment_pipeline(limits_cpu,limits_memory,requests_cpu,requests_memory):
    container_op = env_function_op().add_env_variable(V1EnvVar(name='limits_cpu', value=limits_cpu)).add_env_variable(V1EnvVar(name='limits_memory', value=limits_memory)).add_env_variable(V1EnvVar(name='requests_cpu', value=requests_cpu)).add_env_variable(V1EnvVar(name='requests_memory', value=requests_memory))

In [ ]:
token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(existing_token=token)

In [ ]:
#Specify pipeline argument values
arguments = {"limits_cpu": "500m","limits_memory": "128Mi", "requests_cpu": "250m","requests_memory": "64Mi"}

#Submit a pipeline run
client.create_run_from_pipeline_func(environment_pipeline,arguments=arguments)